In [60]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import data

In [61]:
#btc options data
df=pd.read_csv('data/data.csv', sep=",", index_col=0)
btc_data = df   
#df.drop(columns=['Unnamed: 0', 'interest_rate'], inplace=True)
btc_data = df   

In [62]:
df["Time to Expiration"].describe()

count    634.000000
mean       0.382187
std        0.280726
min        0.079452
25%        0.156164
50%        0.252055
75%        0.654795
max        0.904110
Name: Time to Expiration, dtype: float64

### Reverse BSM approach 

In [63]:
from scipy.stats import norm
import numpy as np

#adjust and set to any 
interest_rate = 0.0150

# Define the Black-Scholes formula for option pricing
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """Calculate Black-Scholes option price for a given option type (call or put)."""
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be either 'call' or 'put'")
    return price

# Define a function to calculate implied volatility by minimizing the error between market and theoretical price
from scipy.optimize import brentq

def implied_volatility(price, S, K, T, r, option_type='call'):
    """Calculate implied volatility using the Black-Scholes model."""
    def objective_function(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - price
    
    try:
        iv = brentq(objective_function, 1e-6, 4.0)  # bounds to search for volatility
    except ValueError:
        iv = np.nan  # if it doesn't converge
    return iv

# Preparing data for calculation
# Extracting necessary columns: underlying_price, Strike Price, Time to Expiration, interest_rate, mark_price
btc_data['BSM_implied_volatility'] = btc_data.apply(
    lambda row: implied_volatility(
        price=row['mark_price'],
        S=row['underlying_price'],
        K=row['Strike Price'],
        T=row['Time to Expiration'],
        r=interest_rate,
        option_type='call'  # Assuming call options for simplicity; adjust as needed.
    ), axis=1
)

#### Volatility Surface

In [64]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata

# Define the columns needed from your data
strikes = btc_data['Strike Price'].values
times_to_expiration = btc_data['Time to Expiration'].values
implied_vols = btc_data['BSM_implied_volatility'].values

# Create meshgrid for strikes and times_to_expiration
X, Y = np.meshgrid(np.unique(strikes), np.unique(times_to_expiration))

# Interpolate the implied volatilities to fill the grid
Z = griddata((strikes, times_to_expiration), implied_vols, (X, Y), method='linear')

# Create the interactive 3D surface plot with color scale title
fig = go.Figure(data=[go.Surface(
    z=Z, x=X, y=Y, colorscale='RdYlGn_r',
    colorbar=dict(title="I.V. %")  # Title for the color scale
)])

fig.update_layout(
    title='Implied Volatility Surface (BSM Approach)',
    autosize=False,
    width=700,
    height=700,
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Expiry (Years)',
        zaxis_title='Implied Volatility %',
        xaxis=dict(type="log"),
        aspectmode="cube"  # Ensures equal aspect ratio for x, y, and z
    )
)

fig.show()


### Vega-Based Iterative Approach  

In [65]:
import numpy as np
from scipy.stats import norm

# Define functions for the Black-Scholes model, Vega, and implied volatility calculation


#adjust and set to any 
interest_rate = 0.05

def black_scholes_price(S, K, T, r, sigma, option_type="call"):
    """
    Calculate Black-Scholes option price for call or put options.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
        
    return price

def vega(S, K, T, r, sigma):
    """
    Calculate Vega, the sensitivity of the option price to changes in volatility.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

def implied_volatility(market_price, S, K, T, r, initial_vol, option_type="call", tolerance=1e-5, max_iterations=100):
    """
    Calculate implied volatility using the Newton-Raphson method.
    """
    sigma = initial_vol
    for i in range(max_iterations):
        price = black_scholes_price(S, K, T, r, sigma, option_type)
        vega_value = vega(S, K, T, r, sigma)
        
        # If vega is very small, avoid division by zero or insignificant changes
        if vega_value < 1e-5:
            break
        
        # Update sigma
        price_difference = market_price - price
        sigma += price_difference / vega_value
        
        # Check for convergence
        if abs(price_difference) < tolerance:
            return sigma
    
    # Return None if no convergence (indicating potential calculation issue)
    return None

# Apply the strike price filter (73% - 120% of spot price)
min_strike_percent = 0.73
max_strike_percent = 1.20

# Filter data within the specified strike price range
filtered_data = btc_data[(btc_data['Strike Price'] >= btc_data['underlying_price'] * min_strike_percent) &
                         (btc_data['Strike Price'] <= btc_data['underlying_price'] * max_strike_percent)]


# Apply the implied volatility function to the data
results = []
for index, row in btc_data.iterrows():
    S = row['underlying_price']
    K = row['Strike Price']
    T = row['Time to Expiration']
    r = interest_rate
    market_price = row['mark_price']
    initial_vol = row['mark_iv'] / 100  # Assuming mark_iv is in percentage

    # Calculate implied volatility
    iv = implied_volatility(market_price, S, K, T, r, initial_vol, option_type="call")
    results.append(iv)

# Add the results to the DataFrame
btc_data['Vega_implied_volatility'] = results

### test 

In [66]:
# Set interest rate
interest_rate = 0.05

# Adjusted implied volatility function that uses the provided vega from the dataset
def implied_volatility_with_dataset_vega(market_price, S, K, T, r, initial_vol, vega_value, option_type="call", tolerance=1e-5, max_iterations=100):
    """
    Calculate implied volatility using the Newton-Raphson method with provided Vega.
    """
    sigma = initial_vol
    for _ in range(max_iterations):
        price = black_scholes_price(S, K, T, r, sigma, option_type)
        
        # Avoid division by very small vega values or insignificant changes
        if vega_value < 1e-5:
            break
        
        # Update sigma based on market price difference and provided vega
        price_difference = market_price - price
        sigma += price_difference / vega_value
        
        # Check for convergence
        if abs(price_difference) < tolerance:
            return sigma
    
    # Return None if no convergence
    return None

# Apply the strike price filter (73% - 120% of spot price)
min_strike_percent = 0.73
max_strike_percent = 1.20

# Filter data within the specified strike price range
filtered_data = btc_data[(btc_data['Strike Price'] >= btc_data['underlying_price'] * min_strike_percent) &
                         (btc_data['Strike Price'] <= btc_data['underlying_price'] * max_strike_percent)]

# List to store the results of implied volatilities
implied_vol_results = []

# Iterate through the filtered data and calculate implied volatility using greeks.vega from the dataset
for _, row in filtered_data.iterrows():
    S = row['underlying_price']
    K = row['Strike Price']
    T = row['Time to Expiration']
    r = interest_rate
    market_price = row['mark_price']
    initial_vol = row['mark_iv'] / 100  # Convert mark_iv to decimal
    vega_value = row['greeks.vega']  # Use dataset's Vega value
    
    # Calculate implied volatility
    iv = implied_volatility_with_dataset_vega(market_price, S, K, T, r, initial_vol, vega_value, option_type="call")
    implied_vol_results.append(iv)

# Add the results to the DataFrame
filtered_data['Vega_implied_volatility'] = implied_vol_results


C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_15764\2332517913.py:54: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [67]:
btc_data = filtered_data
# Define the columns needed from your data
strikes = btc_data['Strike Price'].values
times_to_expiration = btc_data['Time to Expiration'].values
implied_vols = btc_data['Vega_implied_volatility'].values

# Create meshgrid for strikes and times_to_expiration
X, Y = np.meshgrid(np.unique(strikes), np.unique(times_to_expiration))

# Interpolate the implied volatilities to fill the grid
Z = griddata((strikes, times_to_expiration), implied_vols, (X, Y), method='linear')

# Create the interactive 3D surface plot with color scale title
fig = go.Figure(data=[go.Surface(
    z=Z, x=X, y=Y, colorscale='RdYlGn_r',  # Green-to-red color scale
    colorbar=dict(title="Implied Volatility %")
)])


fig.update_layout(
    title='Implied Volatility Surface (Vega Approach)',
    autosize=False,
    width=700,
    height=700,
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Expiry (Years)',
        zaxis_title='Implied Volatility %',
        xaxis=dict(type="log"),
        aspectmode="cube"  # Ensures equal aspect ratio for x, y, and z
    )
)

fig.show()


In [68]:
### endtest

#### Volatility Surface

In [69]:

# Define the columns needed from your data
strikes = btc_data['Strike Price'].values
times_to_expiration = btc_data['Time to Expiration'].values
implied_vols = btc_data['Vega_implied_volatility'].values

# Create meshgrid for strikes and times_to_expiration
X, Y = np.meshgrid(np.unique(strikes), np.unique(times_to_expiration))

# Interpolate the implied volatilities to fill the grid
Z = griddata((strikes, times_to_expiration), implied_vols, (X, Y), method='linear')

# Create the interactive 3D surface plot with color scale title
fig = go.Figure(data=[go.Surface(
    z=Z, x=X, y=Y, colorscale='RdYlGn_r',  # Green-to-red color scale
    colorbar=dict(title="Implied Volatility %")
)])


fig.update_layout(
    title='Implied Volatility Surface (Vega Approach)',
    autosize=False,
    width=700,
    height=700,
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Expiry (Years)',
        zaxis_title='Implied Volatility %',
        xaxis=dict(type="log"),
        aspectmode="cube"  # Ensures equal aspect ratio for x, y, and z
    )
)

fig.show()


### Debribit Mark_IV Volatility Surface

In [70]:
# Converting "mark_iv" column to a percentage scale between 0 and 1 by dividing by 100
df['mark_iv_percentage'] = df['mark_iv'] / 100

In [71]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata

# Define the spot price and percentage limits
spot_price = df['underlying_price'].mean()  # Assuming a spot price column exists

# Extract relevant data for the surface plot
strikes = df['Strike Price'].values
times_to_expiration = df['Time to Expiration'].values
implied_vols = df['mark_iv_percentage'].values

# Create meshgrid for strikes and times_to_expiration
X, Y = np.meshgrid(np.unique(strikes), np.unique(times_to_expiration))

# Interpolate the implied volatilities to fill the grid
Z = griddata((strikes, times_to_expiration), implied_vols, (X, Y), method='linear')

# Create the interactive 3D surface plot
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')])

fig.update_layout(title='Implied Volatility Surface Debribit Mark_IV', autosize=False,
                  width=700, height=700,
                  scene=dict(
                      xaxis_title='Strike Price',
                      yaxis_title='Time to Expiry',
                      zaxis_title='Implied Volatility'))
fig.show()
